In [2]:
import pandas as pd
import spacy
import da_core_news_sm
import re

df_sentences = pd.read_csv('../data/sentences.csv')


In [ ]:
# text list from the worst
# texts = worst.sentence.tolist()
# tokens = nlp(texts[2])
# tokens = [token.text for token in tokens]
# #tokens
# bert.predict(tokens, IOBformat=False)

# extract named entities from the worst sentences
# import spacy
# import da_core_news_sm
# nlp = da_core_news_sm.load()

nlp = da_core_news_sm.load()

def get_named_entities(text):
    """Extract named entities from text"""
    # trim for whitespace, Bert does not like trailing whitespace. At all.
    text = re.sub(r'\x95', '', text)
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)

    doc = nlp(text)
    tokens = [token.text for token in doc]
    preds = bert.predict(tokens, IOBformat=False)
    return [(ent['text'], ent['type']) for ent in preds['entities']]

In [ ]:
collector =[]# get named entities for each sentence
bad_sentences=[]
for index, row in df_sentences.iterrows():
    text = row['sentence'].strip()
    try:
        ents = get_named_entities(text)
        for ent in ents:
            collector.append({'sentence_id': index, 'text': ent[0], 'type': ent[1]})
    except Exception as e:
        bad_sentences.append({'sentence_id': index, 'text': text})


In [5]:
from transformers import pipeline

ner = pipeline(task='ner', 
               model='saattrupdan/nbailab-base-ner-scandi', 
               aggregation_strategy='first')

In [6]:
def get_named_entities_with_scandi(text):
    """Extract named entities from text"""
    # trim for whitespace, Bert does not like trailing whitespace. At all.
    text = re.sub(r'\x95', '', text)
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)

#    doc = nlp(text)
#    tokens = [token.text for token in doc]
    preds = ner(text)
    return [(ent['word'], ent['entity_group']) for ent in preds]

In [15]:
import re
df_sentences = pd.read_csv('../data/sentences.csv')
collector =[]# get named entities for each sentence
bad_sentences=[]
for index, row in df_sentences.iterrows():
    text = row['sentence']
    ents = get_named_entities_with_scandi(text)
    for ent in ents:
        collector.append({'sentence_id': index, 'text': ent[0], 'type': ent[1]})


TypeError: expected string or bytes-like object

In [12]:
df_sentences.shape

(14234, 4)

In [ ]:
df_ents = pd.DataFrame(collector)
df_ents.to_csv('NER_entities_scandi.csv')


In [4]:
import pandas as pd
df_ents = pd.read_csv('../data/NER_entities.csv')
# group and count per text and type
grouped = df_ents.groupby(['text', 'type']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)

grouped.to_csv('../data/NER_entities_grouped.csv')


